In [ ]:
!pip install opendatasets --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"karmakarabhishek5","key":"36b4a526001d0aea0dbb0308eec30991"}'}

In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d uciml/electric-power-consumption-data-set

electric-power-consumption-data-set.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip electric-power-consumption-data-set.zip

Archive:  electric-power-consumption-data-set.zip
replace household_power_consumption.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install elephas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('household_power_consumption.txt', sep=';', 
                 parse_dates={'dt' : ['Date', 'Time']}, infer_datetime_format=True, 
                 low_memory=False, na_values=['nan','?'], index_col='dt')

In [ ]:
len(df)

2075259

In [ ]:
import numpy as np
for cols in df.iloc[:,2:].columns:
    print(cols)
    for x in [cols]:
        q75,q25 = np.percentile(df.loc[:,x],[75,25])
        intr_qr = q75-q25
        
        max1 = q75+(1.5*intr_qr)
        min1 = q25-(1.5*intr_qr)
        
        df.loc[df[x] < min1,x] = np.NaN
        df.loc[df[x] > max1,x] = np.NaN

# sns.heatmap(df.isnull(), yticklabels=False, cmap='viridis')    



df = df.fillna(df.mean())
df.isnull().sum()


df = df[['Global_active_power', 'Global_reactive_power', 'Voltage',
       'Global_intensity', 'Sub_metering_2', 'Sub_metering_1','Sub_metering_3']]



# We will frame the supervised learning problem as predicting the Global_active_power at the current time (t) 
# given the Global_active_power measurement and other features at the prior time step

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    dff = pd.DataFrame(data)
    cols, names = list(), list()
    for i in range(n_in, 0, -1):
        cols.append(dff.shift(-i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    for i in range(0, n_out):
        cols.append(dff.shift(-i))
        if i==0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1)) for j in range(n_vars)]        
        agg = pd.concat(cols, axis=1)
        agg.columns = names
        if dropnan:
            agg.dropna(inplace=True)
        return agg

df_resample = df.resample('h').mean() 
df_resample.shape


from sklearn.preprocessing import MinMaxScaler

values = df_resample.values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
reframed = series_to_supervised(scaled, 1, 1)
r = list(range(df_resample.shape[1]+1, 2*df_resample.shape[1]))
reframed.drop(reframed.columns[r], axis=1, inplace=True)
reframed.head()




# Data spliting into train and test data series. Only 30000 first data points are selected for traing purpose.
values = reframed.values
n_train_time = 30000
train = values[:n_train_time, :]
test = values[n_train_time:, :]

# split into input and outputs
train_x, train_y = train[:, :-1], train[:, -1]
test_x, test_y = test[:, :-1], test[:, -1]

# reshape input to be 3D [samples, timesteps, features]
# We reshaped the input into the 3D format as expected by LSTMs, namely [samples, timesteps, features]
train_x = train_x.reshape((train_x.shape[0], 1, train_x.shape[1]))
test_x = test_x.reshape((test_x.shape[0], 1, test_x.shape[1]))


Voltage
Global_intensity
Sub_metering_1
Sub_metering_2
Sub_metering_3


# Global Active Power

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from elephas.spark_model import SparkModel
from elephas.utils.rdd_utils import to_simple_rdd

# Creating Spark context
master_url = 'local[8]'
conf = SparkConf().setAppName("IOT_LSTM_Elephas_App")
conf.setMaster(master_url)
sc = SparkContext.getOrCreate(conf = conf)

model1 = Sequential()
model1.add(LSTM(100, input_shape=(train_x.shape[1], train_x.shape[2])))
# model1.add(Dropout(0.1))
model1.add(Dense(30))
model1.add(Dense(1))
model1.compile(loss='mean_squared_error', optimizer='adam')

# Convert TrainX and TrainY to RDD
rdd = to_simple_rdd(sc, train_x, train_y)

# Create Spark model
spark_model = SparkModel(model1, frequency='epoch', mode='asynchronous', num_workers=1)
spark_model.fit(rdd, epochs=25, batch_size=70, validation_data=(test_x, test_y), verbose=2, shuffle=False)

# Evaluate Spark model by evaluating the underlying model
score = spark_model.master_network.evaluate(test_x, test_y)

print('Test Loss:', score)

>>> Fit model
 * Serving Flask app "elephas.parameter.server" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://172.28.0.2:4000/ (Press CTRL+C to quit)


>>> Initialize workers
>>> Distribute load


INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 17:58:45] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 17:58:49] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 17:58:49] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 17:58:52] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 17:58:52] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 17:58:55] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 17:58:55] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 17:58:57] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 17:58:57] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 17:59:00] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 17:59:00] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 17:59:02] "POST /update HTTP/1.1" 200 

>>> Async training complete.
144/144 [==============================] - 1s 2ms/step - loss: 0.0063
Test Loss: 0.006256712134927511


# **Submetering_3**

In [ ]:
df = df[['Sub_metering_3','Global_active_power', 'Global_reactive_power', 'Voltage',
       'Global_intensity', 'Sub_metering_2', 'Sub_metering_1']]

from sklearn.preprocessing import MinMaxScaler

values = df_resample.values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
reframed = series_to_supervised(scaled, 1, 1)
r = list(range(df_resample.shape[1]+1, 2*df_resample.shape[1]))
reframed.drop(reframed.columns[r], axis=1, inplace=True)
reframed.head()

# Data spliting into train and test data series. Only 4000 first data points are selected for traing purpose.
values = reframed.values
n_train_time = 30000
train = values[:n_train_time, :]
test = values[n_train_time:, :]
train_x, train_y = train[:, :-1], train[:, -1]
test_x, test_y = test[:, :-1], test[:, -1]
train_x = train_x.reshape((train_x.shape[0], 1, train_x.shape[1]))
test_x = test_x.reshape((test_x.shape[0], 1, test_x.shape[1]))

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from elephas.spark_model import SparkModel
from elephas.utils.rdd_utils import to_simple_rdd

# Creating Spark context
master_url = 'local[8]'
conf = SparkConf().setAppName("IOT_LSTM_Elephas_App")
conf.setMaster(master_url)
sc = SparkContext.getOrCreate(conf = conf)

model7 = Sequential()
model7.add(LSTM(100, input_shape=(train_x.shape[1], train_x.shape[2])))
model7.add(Dropout(0.1))
model7.add(Dense(1))
model7.compile(loss='mean_squared_error', optimizer='adam')


# Convert TrainX and TrainY to RDD
rdd = to_simple_rdd(sc, train_x, train_y)

# Create Spark model
spark_model = SparkModel(model7, frequency='epoch', mode='asynchronous', num_workers=1)
spark_model.fit(rdd, epochs=40, batch_size=100, validation_data=(test_x, test_y), verbose=2, shuffle=False)

# Evaluate Spark model by evaluating the underlying model
score = spark_model.master_network.evaluate(test_x, test_y)

print('Test Loss:', score)


>>> Fit model
 * Serving Flask app "elephas.parameter.server" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://172.28.0.2:4000/ (Press CTRL+C to quit)


>>> Initialize workers
>>> Distribute load


INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 17:59:49] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 17:59:52] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 17:59:52] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 17:59:53] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 17:59:53] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 17:59:54] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 17:59:54] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 17:59:56] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 17:59:56] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 17:59:57] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 17:59:57] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 17:59:58] "POST /update HTTP/1.1" 200 

>>> Async training complete.
144/144 [==============================] - 1s 2ms/step - loss: 0.0063
Test Loss: 0.006322259549051523


# **Global Reactive Power**

In [ ]:
df = pd.read_csv('household_power_consumption.txt', sep=';', 
                 parse_dates={'dt' : ['Date', 'Time']}, infer_datetime_format=True, 
                 low_memory=False, na_values=['nan','?'], index_col='dt')

# sns.heatmap(df.isnull(), yticklabels=False, cmap='viridis')

len(df)

import numpy as np
for cols in df.iloc[:,2:].columns:
    print(cols)
    for x in [cols]:
        q75,q25 = np.percentile(df.loc[:,x],[75,25])
        intr_qr = q75-q25
        
        max1 = q75+(1.5*intr_qr)
        min1 = q25-(1.5*intr_qr)
        
        df.loc[df[x] < min1,x] = np.NaN
        df.loc[df[x] > max1,x] = np.NaN

# sns.heatmap(df.isnull(), yticklabels=False, cmap='viridis')    



df = df.fillna(df.mean())
df.isnull().sum()



df = df[['Global_reactive_power', 'Sub_metering_3','Global_active_power',  'Voltage',
       'Global_intensity', 'Sub_metering_2', 'Sub_metering_1']]

df_resample = df.resample('h').mean() 
df_resample.shape 

from sklearn.preprocessing import MinMaxScaler

# Scalarization
values = df_resample.values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
reframed = series_to_supervised(scaled, 1, 1)
r = list(range(df_resample.shape[1]+1, 2*df_resample.shape[1]))
reframed.drop(reframed.columns[r], axis=1, inplace=True)
reframed.head()

# Data spliting into train and test data series. Only 4000 first data points are selected for traing purpose.
values = reframed.values
n_train_time = 30000
train = values[:n_train_time, :]
test = values[n_train_time:, :]
train_x, train_y = train[:, :-1], train[:, -1]
test_x, test_y = test[:, :-1], test[:, -1]
train_x = train_x.reshape((train_x.shape[0], 1, train_x.shape[1]))
test_x = test_x.reshape((test_x.shape[0], 1, test_x.shape[1]))


from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from elephas.spark_model import SparkModel
from elephas.utils.rdd_utils import to_simple_rdd


model2 = Sequential()
model2.add(LSTM(100, input_shape=(train_x.shape[1], train_x.shape[2])))
# model2.add(Dropout(0.1))
model2.add(Dense(30))
model2.add(Dense(1))
model2.compile(loss='mean_squared_error', optimizer='adam')

# Convert TrainX and TrainY to RDD
rdd = to_simple_rdd(sc, train_x, train_y)

# Create Spark model
spark_model = SparkModel(model2, frequency='epoch', mode='asynchronous', num_workers=1)
spark_model.fit(rdd, epochs=20, batch_size=100, validation_data=(test_x, test_y), verbose=2, shuffle=False)

# Evaluate Spark model by evaluating the underlying model
score = spark_model.master_network.evaluate(test_x, test_y)
print('Test Loss:', score)


Voltage
Global_intensity
Sub_metering_1
Sub_metering_2
Sub_metering_3
>>> Fit model
 * Serving Flask app "elephas.parameter.server" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://172.28.0.2:4000/ (Press CTRL+C to quit)


>>> Initialize workers
>>> Distribute load


INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:01:00] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:01:03] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:01:03] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:01:05] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:01:05] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:01:06] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:01:06] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:01:07] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:01:07] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:01:09] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:01:09] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:01:10] "POST /update HTTP/1.1" 200 

>>> Async training complete.
144/144 [==============================] - 1s 2ms/step - loss: 0.0054
Test Loss: 0.005359277594834566


# **Predict Voltage**

In [ ]:
df = df[['Voltage', 'Global_active_power',  'Global_reactive_power', 'Global_intensity', 'Sub_metering_2', 'Sub_metering_1','Sub_metering_3']]

df_resample = df.resample('h').mean() 
df_resample.shape 

from sklearn.preprocessing import MinMaxScaler

values = df_resample.values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
reframed = series_to_supervised(scaled, 1, 1)
r = list(range(df_resample.shape[1]+1, 2*df_resample.shape[1]))
reframed.drop(reframed.columns[r], axis=1, inplace=True)
reframed.head()

# Data spliting into train and test data series. Only 4000 first data points are selected for traing purpose.
values = reframed.values
n_train_time = 30000
train = values[:n_train_time, :]
test = values[n_train_time:, :]
train_x, train_y = train[:, :-1], train[:, -1]
test_x, test_y = test[:, :-1], test[:, -1]
train_x = train_x.reshape((train_x.shape[0], 1, train_x.shape[1]))
test_x = test_x.reshape((test_x.shape[0], 1, test_x.shape[1]))

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from elephas.spark_model import SparkModel
from elephas.utils.rdd_utils import to_simple_rdd

model3 = Sequential()
model3.add(LSTM(100, input_shape=(train_x.shape[1], train_x.shape[2])))
# model3.add(Dropout(0.1))
model3.add(Dense(1))
model3.compile(loss='mean_squared_error', optimizer='adam')

# Convert TrainX and TrainY to RDD
rdd = to_simple_rdd(sc, train_x, train_y)

# Create Spark model
spark_model = SparkModel(model3, frequency='epoch', mode='asynchronous', num_workers=1)
spark_model.fit(rdd, epochs=13, batch_size=100, validation_data=(test_x, test_y), verbose=2, shuffle=False)

# Evaluate Spark model by evaluating the underlying model
score = spark_model.master_network.evaluate(test_x, test_y)
print('Test Loss:', score)

>>> Fit model
 * Serving Flask app "elephas.parameter.server" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
>>> Initialize workers
>>> Distribute load


INFO:werkzeug: * Running on http://172.28.0.2:4000/ (Press CTRL+C to quit)
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:01:36] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:01:40] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:01:40] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:01:41] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:01:41] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:01:42] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:01:42] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:01:44] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:01:44] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:01:45] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:01:45] "GET /parameters HTTP/1.1" 200 -
INFO:

>>> Async training complete.
144/144 [==============================] - 1s 2ms/step - loss: 0.0031
Test Loss: 0.003096235916018486


# **Global Intensity**

In [ ]:
df = df[['Global_intensity', 'Global_active_power',  'Global_reactive_power', 'Voltage', 'Sub_metering_2', 'Sub_metering_1','Sub_metering_3']]

df_resample = df.resample('h').mean() 
df_resample.shape 

from sklearn.preprocessing import MinMaxScaler

values = df_resample.values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
reframed = series_to_supervised(scaled, 1, 1)
r = list(range(df_resample.shape[1]+1, 2*df_resample.shape[1]))
reframed.drop(reframed.columns[r], axis=1, inplace=True)
reframed.head()

# Data spliting into train and test data series. Only 4000 first data points are selected for traing purpose.
values = reframed.values
n_train_time = 30000
train = values[:n_train_time, :]
test = values[n_train_time:, :]
train_x, train_y = train[:, :-1], train[:, -1]
test_x, test_y = test[:, :-1], test[:, -1]
train_x = train_x.reshape((train_x.shape[0], 1, train_x.shape[1]))
test_x = test_x.reshape((test_x.shape[0], 1, test_x.shape[1]))


from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from elephas.spark_model import SparkModel
from elephas.utils.rdd_utils import to_simple_rdd


model4 = Sequential()
model4.add(LSTM(100, input_shape=(train_x.shape[1], train_x.shape[2])))
# model3.add(Dropout(0.1))
model4.add(Dense(30))
model4.add(Dense(1))
model4.compile(loss='mean_squared_error', optimizer='adam')

# Convert TrainX and TrainY to RDD
rdd = to_simple_rdd(sc, train_x, train_y)

# Create Spark model
spark_model = SparkModel(model4, frequency='epoch', mode='asynchronous', num_workers=1)
spark_model.fit(rdd, epochs=13, batch_size=100, validation_data=(test_x, test_y), verbose=2, shuffle=False)

# Evaluate Spark model by evaluating the underlying model
score = spark_model.master_network.evaluate(test_x, test_y)
print('Test Loss:', score)

>>> Fit model
 * Serving Flask app "elephas.parameter.server" (lazy loading)
 * Environment: production
>>> Initialize workers
>>> Distribute load
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://172.28.0.2:4000/ (Press CTRL+C to quit)
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:01:59] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:02:02] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:02:02] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:02:05] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:02:05] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:02:06] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:02:06] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:02:07] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:02:07] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:02:08] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:02:08] "GET /parameters HTTP/1.1" 200 -
INFO:

>>> Async training complete.
144/144 [==============================] - 1s 2ms/step - loss: 0.0058
Test Loss: 0.005831320304423571


# **Submetering_1**

In [ ]:
df = df[['Sub_metering_1', 'Global_active_power',  'Global_reactive_power', 'Voltage', 'Global_intensity', 'Sub_metering_2','Sub_metering_3']]

df_resample = df.resample('h').mean() 
df_resample.shape 

from sklearn.preprocessing import MinMaxScaler

values = df_resample.values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
reframed = series_to_supervised(scaled, 1, 1)
r = list(range(df_resample.shape[1]+1, 2*df_resample.shape[1]))
reframed.drop(reframed.columns[r], axis=1, inplace=True)
reframed.head()

# Data spliting into train and test data series. Only 4000 first data points are selected for traing purpose.
values = reframed.values
n_train_time = 30000
train = values[:n_train_time, :]
test = values[n_train_time:, :]
train_x, train_y = train[:, :-1], train[:, -1]
test_x, test_y = test[:, :-1], test[:, -1]
train_x = train_x.reshape((train_x.shape[0], 1, train_x.shape[1]))
test_x = test_x.reshape((test_x.shape[0], 1, test_x.shape[1]))


from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from elephas.spark_model import SparkModel
from elephas.utils.rdd_utils import to_simple_rdd

model5 = Sequential()
model5.add(LSTM(200, input_shape=(train_x.shape[1], train_x.shape[2])))
model5.add(Dropout(0.5))
model5.add(Dense(1))
model5.compile(loss='mean_squared_error', optimizer='adam')


# Convert TrainX and TrainY to RDD
rdd = to_simple_rdd(sc, train_x, train_y)

# Create Spark model
spark_model = SparkModel(model5, frequency='epoch', mode='asynchronous', num_workers=1)
spark_model.fit(rdd, epochs=50, batch_size=100, validation_data=(test_x, test_y), verbose=2, shuffle=False)

# Evaluate Spark model by evaluating the underlying model
score = spark_model.master_network.evaluate(test_x, test_y)
print('Test Loss:', score)

>>> Fit model
 * Serving Flask app "elephas.parameter.server" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://172.28.0.2:4000/ (Press CTRL+C to quit)


>>> Initialize workers
>>> Distribute load


INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:02:24] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:02:28] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:02:28] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:02:30] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:02:30] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:02:32] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:02:32] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:02:35] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:02:35] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:02:37] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:02:37] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:02:39] "POST /update HTTP/1.1" 200 

>>> Async training complete.
144/144 [==============================] - 1s 2ms/step - loss: 0.0032
Test Loss: 0.003230471396818757


# **Submetering_2**

In [ ]:
df = df[['Sub_metering_2', 'Global_active_power',  'Global_reactive_power', 'Voltage', 'Global_intensity', 'Sub_metering_1','Sub_metering_3']]

df_resample = df.resample('h').mean() 
df_resample.shape 

from sklearn.preprocessing import MinMaxScaler

values = df_resample.values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
reframed = series_to_supervised(scaled, 1, 1)
r = list(range(df_resample.shape[1]+1, 2*df_resample.shape[1]))
reframed.drop(reframed.columns[r], axis=1, inplace=True)
reframed.head()

# Data spliting into train and test data series. Only 4000 first data points are selected for traing purpose.
values = reframed.values
n_train_time = 30000
train = values[:n_train_time, :]
test = values[n_train_time:, :]
train_x, train_y = train[:, :-1], train[:, -1]
test_x, test_y = test[:, :-1], test[:, -1]
train_x = train_x.reshape((train_x.shape[0], 1, train_x.shape[1]))
test_x = test_x.reshape((test_x.shape[0], 1, test_x.shape[1]))


from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from elephas.spark_model import SparkModel
from elephas.utils.rdd_utils import to_simple_rdd

model6 = Sequential()
model6.add(LSTM(100, input_shape=(train_x.shape[1], train_x.shape[2])))
# model3.add(Dropout(0.5))
model6.add(Dense(1))
model6.compile(loss='mean_squared_error', optimizer='RMSprop')

# Convert TrainX and TrainY to RDD
rdd = to_simple_rdd(sc, train_x, train_y)

# Create Spark model
spark_model = SparkModel(model5, frequency='epoch', mode='asynchronous', num_workers=1)
spark_model.fit(rdd, epochs=50, batch_size=100, validation_data=(test_x, test_y), verbose=2, shuffle=False)

# Evaluate Spark model by evaluating the underlying model
score = spark_model.master_network.evaluate(test_x, test_y)
print('Test Loss:', score)

>>> Fit model
 * Serving Flask app "elephas.parameter.server" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://172.28.0.2:4000/ (Press CTRL+C to quit)


>>> Initialize workers
>>> Distribute load


INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:04:27] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:04:33] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:04:33] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:04:36] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:04:36] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:04:38] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:04:38] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:04:42] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:04:42] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:04:45] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:04:45] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [16/Nov/2022 18:04:48] "POST /update HTTP/1.1" 200 

>>> Async training complete.
144/144 [==============================] - 1s 4ms/step - loss: 0.0031
Test Loss: 0.0031452171970158815
